In [66]:
import numpy as np
from PIL import Image
from make_gts import color_map
import os
import re

In [57]:
cmap = color_map('fundus')

In [28]:
def compute_padding(length):
    length_padding = 512-length
    left_bottom_padding = np.around(length_padding/2).astype(np.uint8)
    right_top_padding = left_bottom_padding
    if not length_padding % 2:
        right_top_padding += 1
    return left_bottom_padding,right_top_padding

def crop(img_path,mask_path):
    mask_arr = np.array(Image.open(mask_path))
    img_arr = np.array(Image.open(img_path))
    shape = mask_arr.shape
    mask_lt0= mask_arr > 0
    mask_lt0_indexs  = np.where(mask_lt0)
    width_idx,height_idx = mask_lt0_indexs[0],mask_lt0_indexs[1]
    top,bottom = width_idx.min(),width_idx.max()
    left,right = height_idx.min(),height_idx.max()

    width = right-left
    height = bottom-top
    left_padding,right_padding = compute_padding(width)
    bottom_padding,top_padding = compute_padding(height)
    # 开始裁剪图片
    new_left = left-left_padding
    new_right = right+right_padding-1
    new_bottom = bottom+bottom_padding-1
    new_top = top-top_padding

    if new_left < 0:
        new_right = new_right+(0-new_left)
        new_left = 0
    if new_right > shape[1]:
        new_left = new_left - (new_right-shape[1])
        new_right = shape[1]
    if new_top < 0:
        new_bottom = new_bottom+(0-new_top)
        new_top = 0
    if new_bottom > shape[0]:
        new_top = new_top-(new_bottom-shape[0])
        new_bottom = shape[0]


    cropped_mask_arr = mask_arr[new_top:new_bottom,new_left:new_right]
    cropped_img_arr = img_arr[new_top:new_bottom,new_left:new_right]

    cropped_img = Image.fromarray(cropped_img_arr)
    cropped_mask = Image.fromarray(cropped_mask_arr,mode='P')
    cropped_mask.putpalette(cmap)
    return cropped_img,cropped_mask

In [65]:
root = '/root/autodl-tmp/data/SEG'
whole_path = '/root/autodl-tmp/Semi4FundusODOC/dataset/SEG/SEG.txt'
whole_cropped_path = '/root/autodl-tmp/Semi4FundusODOC/dataset/SEG/SEG_cropped.txt'
with open(whole_path, 'r') as f:
    whole_ids = f.read().splitlines()
with open(whole_cropped_path,'a') as cf:
    for id in whole_ids:
        img_path = os.path.join(root, id.split(' ')[0])
        mask_path = os.path.join(root, id.split(' ')[1])

        cropped_img,cropped_mask = crop(img_path,mask_path)
        mask_path = mask_path.replace('my_gts','my_gts_cropped')
        if 'images' in img_path:
            img_path = img_path.replace('images','images_cropped')
        if 'Images' in img_path:
            img_path = img_path.replace('Images','images_cropped')
        if '650image' in img_path:
            img_path = img_path.replace('650image','images_cropped')
        if 'MESSIDOR' in img_path:
            img_path = img_path.replace('MESSIDOR','MESSIDOR_cropped')
        if 'Magrabia' in img_path:
            img_path = img_path.replace('Magrabia','Magrabia_cropped')
        if 'BinRushed' in img_path:
            img_path = img_path.replace('BinRushed','BinRushed_cropped')
        if 'imgs' in img_path:
            img_path = img_path.replace('imgs','images_cropped')
        image_name = os.path.basename(img_path)
        mask_name = os.path.basename(mask_path)
        img_root = img_path.split(image_name)[0]
        mask_root = mask_path.split(mask_name)[0]
        if not os.path.exists(img_root):
            os.makedirs(img_root)
        if not os.path.exists(mask_root):
            os.makedirs(mask_root)
        mask_name = os.path.basename(mask_path)
        cropped_img_path = os.path.join(img_root,image_name)
        cropped_mask_path = os.path.join(mask_root,mask_name)
        cf.write(cropped_img_path+' '+cropped_mask_path+'\n')

        cropped_img.save(cropped_img_path)
        cropped_mask.save(cropped_mask_path)
print('done')


[0 1 2]
(512, 512)
